## Universidad Austral de Chile - INFO133: Base de Datos

### Responsable: Matthieu Vernier, mvernier@inf.uach.cl

# TP: Introducción al SGBD Mongo

MongoDB es un SGBD creado en 2009 (en 2018: version 4). Es un SGBD que no implementa el modelo Relacional (Tabla/Tupla/Claves, datos atómicos) como MySQL o PostgreSQL. La idea inicial de Mongo es ofrecer una solución simple para almacenar objetos JSON.

En Mongo, los conceptos claves son:
- los <b>Documentos</b>: un documento corresponde a objeto JSON.
- las <b>Colecciones</b>: una colección es un conjunto de documentos. Nota Bene: en una colección Mongo, los documentos pueden tener una estructura distinta.

En Mongo, no existen directamente los conceptos de Tabla, Tupla o Claves primerias/foraneas. Sin embargo, podemos ver una equivalencia entre Tabla y Colección, y entre Tupla y Documento.

Verificar que Mongo está instalado correctamente ejecutando:
- el programa principal del SGBD: <code>mongod &</code> desde un terminal
- el programa CLI (Command Line Interface): <code>mongo</code>

## Utilizar Mongo desde el Terminal

Seguir el Tutorial "Mongo" de TutorialsPoint.com: https://www.tutorialspoint.com/mongodb/index.htm

## Utilizar Mongo desde un entorno Python

Vamos a interactuar con Mongo desde un entorno Python, por lo tanto utilizaremos el driver <code>PyMongo</code> (que instalar).

In [ ]:
import pymongo
from pymongo import MongoClient

#Nos conectamos a la instancia de Mongo local, puerto 27017
client = MongoClient('localhost', 27017)

#Nos conectamos a la BD 'notebook5_db'
db = client['tpmongo_db']

#Dentro de la BD, creamos una nueva colección llamada 'personas'
collection = db['personas']

A continuación, mostramos un ejemplo de como insertar nuevos documentos en una colección:

In [ ]:
#Creamos un nuevo documento

aDocument = {"Nombre":"Juan", "Edad":23, "Asignaturas":["INFO261", "INFO267", "INFO268"] }
otherDocument = {"Nombre":"Maria", "Edad":24, "Asignaturas":["INFO261", "INFO267", "INFO268", "INFO331"] }

collection.insert_many([aDocument,otherDocument])

Un ejemplo de como consultar nuestra colección agregando un filtro para ver solamente los documentos que tienen un campo "Edad"=23.

In [ ]:
from pprint import pprint

for doc in collection.find({"Edad":23}):
    pprint(doc)

Para más detalles sobre la API de Mongo y PyMongo para hacer consultas:
- http://api.mongodb.com/python/current/tutorial.html
- https://docs.mongodb.com/manual/crud/

## Ejercicio 1: Crear una base de datos básica desde el Terminal

1) Creamos una nueva base de datos llamado <b>nb7-peliculas</b>

<code>use peliculas</code>

Deberian ver como respuesta del sistema la respuesta: "switched to db peliculas"

Mongo CLI no utiliza SQL para realizar operaciones CRUD (Create/Read/Update/Delete), utiliza su propia API parecida al lenguaje Javascript.

La variable <code>db</code> representa siempre la base de datos que estamos utilizando (en este caso "nb7-peliculas").

2) Creamos una colección <b>actors</b>

<code>db.actors.insert({firstname:"Will", lastname:"Smith"})</code>

Como pueden verlo, para crear la colección no tuvimos que definir la estructura de la colección, simplemente hemos insertado un documento.

Verificamos que hemos realmente creado la coleccion con:

<code>db.actors.find()</code>

3) Insertamos varios documentos en la colección en un solo comando:

<code>db.actors.insertMany([{firstname:"Scarlett", lastname:"Johanson"},{firstname:"Matt", lastname:"Damon"},{firstname:"Nathan", lastname:"Smith"}])</code>

4) Contamos cuántos documentos tenemos en la colección:

<code>db.actors.count()</code>

Como en SQL, la API de Mongo permite construir consultas para filtrar, ordenar o agrupar datos. Sin embargo, la síntaxis es distinta (y amenudo menos simple e intuitiva que SQL).

5) Filtramos los actores que se llaman 'Smith'

<code>db.actors.find({lastname:"Smith"})</code>

Mongo tambien permite modificar los datos (ojo: Mongo no tiene mecanismo interno para asegurar la coherencia de los datos, es la responsabilidad del programador de la aplicación).

6) Actualizamos un documento. Por ejemplo, "Nathan Smith" se llama ahora "Nathan Lane"

<code>db.actors.update({firstname:"Nathan", lastname:"Smith"},{$set : {lastname:"Lane"}})</code>

Este último comando va a actualizar el primer documento encontrado que corresponde al filtro definido.

7) Supongamos ahora que queremos agregar un campo "gender" en todos los documentos:

¿Qué hace el comando siguiente? ¿Cuál es el limite?

<code>db.actors.update({},{$set : {gender:"M"}})</code>

Para actualizar todos los documentos, y no simplemente el primero, podemos utilizar el comando:

<code>db.actors.update({},{$set : {gender:"M"}},{multi:true})</code>

8) Actualizar el documento "Scarlett Johanson", para indicar que es una mujer (gender:"F").

9) Para suprimir un atributo, podemos utilizar el operador $unset. Por ejemplo:
    
<code>db.actors.update({},{$unset : {gender:1}},{multi:true})</code>

10) Para suprimir un documento completo, podemos utilizar el comando:

<code>db.actors.remove({lastname:"Smith"})</code>

11) Agregamos un atributo de tipo arreglo en un documento

<code>db.actors.update({firstname:"Matt"},{$set : {tab:[1,"a"]}})</code>

12) Añadimos un valor en el arreglo con el operator $push

<code>db.actors.update({firstname:"Matt"},{$push : {tab:["new_value"]}})</code>

¿Cuál es el problema con el último comando? ¿Cuál es la diferencia con el comando siguiente?

<code>db.actors.update({firstname:"Matt"},{$push : {tab:"new_value"}})</code>

Cómo pueden verlo el SGBD no relacional Mongo es mucho más flexible que MySQL, pero deja mucho más posibilidad de equivocarse.


No duden en consultar la documentación de Mongo para probar otros comandos CRUD: https://docs.mongodb.com/manual/crud/

## Ejercicio 2: Primeras consultas avanzadas con una colección de peliculas

En este segundo ejercicio, trabajaremos con una base de datos existente llamada "movie details", la cual tiene 2295 documentos JSON resumiendo información sobre peliculas.

- Descargar el dataset https://raw.githubusercontent.com/steveren/docs-assets/charts-tutorial/movieDetails.json. ¿Qué contiene este archivo? Un documento JSON? Un arreglo de documentos JSON? Una lista de documentos JSON?

- Importamos los datos en Mongo con el comando siguiente:

<code>mongoimport --db peliculas --collection movieDetails --drop --file <path/to/movieDetails.json></code>

A continuación, queremos realizar algunas consultas típicas para observar la diferencia entre el lenguaje de consulta SQL y el lenguaje de Mongo.

<b>Nota bena</b>: ¿Qué pasa si agregan la función <code>.pretty()</code> al final de un comando? Por ejemplo: <code>db.actors.find().pretty()</code>

1) Utilizar un comando Mongo para contar cuántos documentos hay en la colección 'movieDetails'. ¿Cuál sería el equivalente en SQL?

2) Mostrar los títulos (title) y años de producción (year) de todas las peliculas de la colección. ¿A qué clausula SQL correspondría?

3) Contar las peliculas realizadas entre 1990 y 2000

4) ¿Cuáles son los titulos de peliculas que ganaron más de 5 awards?

5) ¿Cuál es el rating promedio en el sitio imdb para todas las peliculas?

6) ¿Cuáles son las peliculas en las cuales participó el actor Nicolas Cage?

7) ¿Cuál es el número de pelicula por año?

8) ¿Cuál es la pelicula con el mejor rating?

9) ¿Cuáles son los thrillers con un rating de al meno 6?

10) ¿Cuáles son todos los valores de generos posibles?

11) ¿Cuáles son las peliculas que contienen la palabra 'Star'?

12) ¿Cuáles son los títulos que empiezan con la letra 'T'?

13) ¿Cuáles son las peliculas que no son de tipo Drama o Thriller?